# 1. Import der benötigten Bibliotheken

In [1]:
# Diese Zelle importiert alle notwendigen Bibliotheken für die Datenanalyse, -bereinigung und Visualisierung.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, f_oneway # Für statistische Tests

# Scikit-learn Bibliotheken für Modellierung und Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Einstellungen für schönere Plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Alle benötigten Bibliotheken wurden erfolgreich importiert.")

Alle benötigten Bibliotheken wurden erfolgreich importiert.


# 2. Daten laden und erste Inspektion

In [ ]:
# Das Notebook liegt im Ordner 'Scripts' und die CSV im Ordner 'Daten'
df = pd.read_csv('../Daten/autoscout24-germany-dataset.csv')

print(f"Datensatz erfolgreich geladen. Form: {df.shape} (Zeilen, Spalten)")
print("\nErste 5 Zeilen des Datensatzes:")
display(df.head()) # 'display' anstelle von 'print' für schönere DataFrame-Ausgabe in Jupyter


print("\nInformationen zu den Datentypen und fehlenden Werten:")
df.info()

print("\nStatistische Zusammenfassung der numerischen Spalten:")
display(df.describe())

# --- Detaillierte erste Einsicht nach dem Laden und Inspizieren ---
print("\n--- Detaillierte erste Einsicht in den Datensatz ---")
print("Basierend auf der ersten Inspektion des Datensatzes lassen sich folgende erste Beobachtungen festhalten:")

print("\n**1. Datenumfang und Struktur:**")
print(f"- Der Datensatz umfasst {df.shape[0]} Einträge (Fahrzeuge) und {df.shape[1]} Spalten (Merkmale).")
print("- Die `head()`-Ausgabe zeigt typische Merkmale von Gebrauchtwagen wie Kilometerstand (`mileage`), Hersteller (`make`), Modell (`model`), Kraftstoff (`fuel`), Getriebeart (`gear`), Angebotstyp (`offerType`), Preis (`price`), PS (`hp`) und Baujahr (`year`).")

print("\n**2. Datentypen und fehlende Werte (`df.info()`):**")
print("- Die meisten Spalten sind als `object` (kategoriale Texte) oder `int64`/`float64` (numerisch) korrekt erkannt worden.")
print("- Es sind erste Anzeichen für **fehlende Werte** erkennbar, insbesondere in Spalten wie `hp` (Horsepower) und `model`. Diese müssen in der Datenbereinigung behandelt werden, da fehlende Werte die meisten Machine-Learning-Modelle stören würden.")
print("- Die Spalte `offerType` scheint fast ausschließlich 'Used' zu enthalten. Dies deutet darauf hin, dass es sich primär um einen Gebrauchtwagenmarkt handelt.")

print("\n**3. Statistische Zusammenfassung der numerischen Spalten (`df.describe()`):**")
print("- **Preis (`price`):**")
print(f"  - Der durchschnittliche Preis liegt bei ca. {df['price'].mean():,.2f} €.")
print(f"  - Die Preisspanne ist sehr breit, von minimal {df['price'].min():,.2f} € bis maximal {df['price'].max():,.2f} €. Dies deutet auf eine große Vielfalt an Fahrzeugen hin, aber auch auf das mögliche Vorhandensein von **Ausreißern**, die in späteren Schritten behandelt werden müssen.")
print(f"  - Die Standardabweichung von {df['price'].std():,.2f} € zeigt eine hohe Variabilität der Preise an.")
print("- **Kilometerstand (`mileage`):**")
print(f"  - Der Kilometerstand variiert von {df['mileage'].min():,} km bis {df['mileage'].max():,} km, mit einem Durchschnitt von ca. {df['mileage'].mean():,.0f} km.")
print(f"  - Der 75. Perzentil liegt bei {df['mileage'].quantile(0.75):,.0f} km, was auf eine Tendenz zu höheren Kilometerständen hinweist.")
print("- **PS (`hp`):**")
print(f"  - Die Leistung reicht von {df['hp'].min():.0f} PS bis {df['hp'].max():.0f} PS. Die Standardabweichung ist relativ hoch, was eine breite Streuung in der Motorisierung der Fahrzeuge impliziert.")
print(f"  - Der Medianwert (50. Perzentil) liegt bei {df['hp'].median():.0f} PS, was im Vergleich zum Maximum darauf hindeutet, dass viele Fahrzeuge im unteren bis mittleren Leistungsbereich liegen.")
print("- **Baujahr (`year`):**")
print(f"  - Die Fahrzeuge stammen aus den Jahren {df['year'].min()} bis {df['year'].max()}. Die Spalte `year` wird später für die Berechnung des Fahrzeugalters (`age`) genutzt werden.")

print("\n**Erster Fazit:**")
print("- Der Datensatz scheint eine solide Basis für die Preisvorhersage zu bieten, erfordert jedoch eine sorgfältige Vorverarbeitung, insbesondere im Umgang mit fehlenden Werten und potenziellen Ausreißern.")
print("- Die breiten Spannen in Preis, Kilometerstand und PS lassen bereits vermuten, dass diese Merkmale starke Prädiktoren für den Fahrzeugpreis sein werden.")
print("\n--- Ende der detaillierten ersten Einsicht ---")

# 3. Datenbereinigung und Feature Engineering

In [ ]:
# Dieser Schritt ist entscheidend, um die Qualität der Daten sicherzustellen und neue, potenziell nützliche Features zu erstellen.
print("\n--- Start der Datenbereinigung und Feature Engineering ---")

## 3.1 Umgang mit doppelten und fehlenden Werten

In [ ]:
# Doppelte Einträge können das Modell verzerren und zu Overfitting führen.
# Fehlende Werte müssen behandelt werden, da die meisten ML-Algorithmen damit nicht umgehen können.
print(f"Anzahl fehlender Werte vor Bereinigung: {df.isnull().sum().sum()}")
print(f"Anzahl doppelter Zeilen vor Bereinigung: {df.duplicated().sum()}")

df.drop_duplicates(inplace=True)
df.dropna(inplace=True) # Wir entfernen Zeilen mit fehlenden Werten für eine saubere Analyse.

In [ ]:
print(f"\nDatensatzform nach Entfernung von Duplikaten und fehlenden Werten: {df.shape}")
print(f"Anzahl fehlender Werte nach Bereinigung: {df.isnull().sum().sum()}")
print(f"Anzahl doppelter Zeilen nach Bereinigung: {df.duplicated().sum()}")

## 3.2 Feature Engineering: Erstellung neuer, nützlicher Merkmale

In [ ]:
# 'age' (Alter des Fahrzeugs): Ein Auto verliert mit dem Alter an Wert.
# Wir berechnen das Alter basierend auf dem aktuellen Jahr (2025).
aktuelles_jahr = 2025 # Stand des Datensatzes ist bis 2021, daher für realitätsnahe Berechnung das Jahr 2025 annehmen.
df['age'] = aktuelles_jahr - df['year']
print(f"\n'age' Spalte hinzugefügt. Max Alter: {df['age'].max()} Jahre, Min Alter: {df['age'].min()} Jahre.")

# 'price_per_mileage' (Preis pro Kilometer): Könnte ein Indikator für den Wert pro Abnutzung sein.
# Vermeide Division durch Null, falls mileage 0 sein könnte (hier nicht erwartet, aber gute Praxis).
# Bei 0 km (Neuwagen/Pre-registered) könnte dies problematisch sein, daher absichern.
df['price_per_mileage'] = df.apply(lambda row: row['price'] / row['mileage'] if row['mileage'] > 0 else row['price'], axis=1)
print("'price_per_mileage' Spalte hinzugefügt.")

## 3.3 Umgang mit Ausreißern (Outlier Treatment)

In [ ]:
# Ausreißer in der Zielvariablen ('price') können das Modelltraining stark beeinflussen.
# Wir verwenden die Interquartilsbereichs-Methode (IQR), um extreme Ausreißer zu identifizieren und zu entfernen.
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
untergrenze_price = Q1 - 1.5 * IQR
obergrenze_price = Q3 + 1.5 * IQR

# Filtere den DataFrame, um Zeilen außerhalb der Ausreißergrenzen zu entfernen
df_bereinigt = df[(df['price'] >= untergrenze_price) & (df['price'] <= obergrenze_price)].copy() # Wichtig: .copy()

print(f"\nAnzahl der entfernten Preis-Ausreißer: {df.shape[0] - df_bereinigt.shape[0]} Zeilen.")
print(f"Datensatzform nach Ausreißer-Bereinigung: {df_bereinigt.shape}")

#Todo: Hier Optional entscheiden

# Optional: Ausreißer bei 'hp' und 'mileage' könnten auch berücksichtigt werden,
# aber für den Anfang konzentrieren wir uns auf 'price'.

# Wir verwenden nun den bereinigten DataFrame für die weitere Analyse
df = df_bereinigt
print("\nDatenbereinigung und Feature Engineering abgeschlossen. Der 'df'-DataFrame wurde aktualisiert.")

# 4. Explorative Datenanalyse (EDA) - Überblick

In [ ]:
# Bevor wir ins Detail gehen, verschaffen wir uns einen Überblick über die Verteilung der Daten
# und erste Korrelationen, insbesondere in Bezug auf den Preis.

print("\n--- Start der Explorativen Datenanalyse (EDA) ---")

## 4.1 Verteilung der Zielvariablen: 'price'

In [ ]:
# Ein Histogramm zeigt uns die Verteilung der Autopreise. Dies hilft zu verstehen,
# ob die Preise eher normalverteilt sind oder Schieflagen aufweisen.
plt.figure(figsize=(10, 6))
sns.histplot(df['price'], bins=50, kde=True, color='skyblue')
plt.title('Verteilung der Autopreise (€)', fontsize=16)
plt.xlabel('Preis (€)', fontsize=14)
plt.ylabel('Anzahl der Fahrzeuge', fontsize=14)
plt.show()

## 4.2 Verteilung der numerischen Features

In [ ]:
# Wir visualisieren die Verteilung von 'mileage', 'hp' und 'age'.
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(df['mileage'], bins=30, kde=True, ax=axes[0], color='lightcoral')
axes[0].set_title('Verteilung des Kilometerstands', fontsize=14)
axes[0].set_xlabel('Kilometerstand', fontsize=12)

sns.histplot(df['hp'], bins=30, kde=True, ax=axes[1], color='lightgreen')
axes[1].set_title('Verteilung der PS (Horsepower)', fontsize=14)
axes[1].set_xlabel('PS', fontsize=12)

sns.histplot(df['age'], bins=30, kde=True, ax=axes[2], color='gold')
axes[2].set_title('Verteilung des Fahrzeugalters', fontsize=14)
axes[2].set_xlabel('Alter (Jahre)', fontsize=12)

plt.tight_layout() # Passt die Plots an, damit sie sich nicht überlappen
plt.show()

## 4.3 Analyse der kategorialen Features

In [ ]:
# Zählung der Vorkommen für 'make', 'fuel', 'gear', 'offerType'
print("\nHäufigkeit der Fahrzeugtypen (Gear):")
display(df['gear'].value_counts())

In [ ]:
## Da ist ein Fehler hier - Das wird nicht angezeigt

print("\nHäufigste Kraftstoffart pro Jahr:")
fuel_year_counts = df.groupby(['year', 'fuel']).size().reset_index(name='count')
# Finde die häufigste Kraftstoffart für jedes Jahr
most_common_fuel_per_year = fuel_year_counts.loc[fuel_year_counts.groupby('year')['count'].idxmax()]
display(most_common_fuel_per_year)

print("\nTop 10 der Hersteller (make) nach Anzahl der Fahrzeuge:")
display(df['make'].value_counts().head(10))

In [ ]:
print("\n--- Explorative Datenanalyse (EDA) abgeschlossen ---")

In [ ]:
____________________________________________________________________________________________________________

 ## 5.1 Top 10 der teuersten Autos auf dem deutschen Automarkt

In [ ]:

# Dieser Abschnitt identifiziert und listet die 10 teuersten Fahrzeuge im Datensatz auf.
# Dies gibt einen schnellen Überblick über die Preissuperlative und die damit verbundenen Marken und Modelle.

print("\n--- 5.1 Top 10 der teuersten Autos auf dem deutschen Automarkt ---")

# Wählen die relevanten Spalten aus und sortiere sie absteigend nach 'price'.
# `head(10)` wählt die obersten 10 Einträge aus.
teuerste_autos = df[['make', 'model', 'price', 'year', 'mileage', 'hp']].sort_values('price', ascending=False).head(10)

print("Die 10 teuersten Autos im Datensatz:")
display(teuerste_autos)

print("\nErkenntnisse:")
print("- Hier sehen wir die Marken und Modelle, die die höchsten Preise erzielen.")
print("- Oftmals handelt es sich hierbei um Luxus- oder Sportwagen mit geringem Kilometerstand und hohem PS-Wert.")
print("- Die Jahre und Kilometerstände geben Aufschluss darüber, wie stark diese Fahrzeuge ihren Wert halten.")

____________________________________________________________________________________________________________

## 5.2 Gesamtzahl der Fahrzeuge je Hersteller

In [ ]:

# Diese Analyse gibt Aufschluss über die Marktpräsenz der verschiedenen Hersteller
# im Datensatz. Hersteller mit einer hohen Anzahl an Einträgen sind in der Regel
# die gängigsten Marken auf dem deutschen Gebrauchtwagenmarkt.

print("\n--- 5.2 Gesamtzahl der Fahrzeuge je Hersteller ---")

# Zähle die Vorkommen jedes Herstellers in der 'make'-Spalte.
# `value_counts()` ist ideal dafür und sortiert die Ergebnisse automatisch absteigend.
fahrzeuge_je_hersteller = df['make'].value_counts()

print("Gesamtzahl der Fahrzeuge je Hersteller (Top 20):")
display(fahrzeuge_je_hersteller.head(20)) # Zeige die Top 20 für Übersichtlichkeit

# Visualisierung der Top N Hersteller
plt.figure(figsize=(15, 8))
sns.barplot(x=fahrzeuge_je_hersteller.head(20).index, y=fahrzeuge_je_hersteller.head(20).values, palette='Blues_d')
plt.title('Top 15 Hersteller nach Anzahl der Fahrzeuge im Datensatz', fontsize=16)
plt.xlabel('Hersteller', fontsize=14)
plt.ylabel('Anzahl der Fahrzeuge', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\nErkenntnisse:")
print("- Dies zeigt, welche Marken im Datensatz am häufigsten vertreten sind, was oft die Beliebtheit am Markt widerspiegelt.")
print("- Hersteller wie Volkswagen, Opel und Ford dominieren typischerweise den deutschen Gebrauchtwagenmarkt.")

____________________________________________________________________________________________________________

## 5.3 Bestes Jahr für den Autoverkauf (nach Anzahl der gelisteten Fahrzeuge)

In [ ]:

# Diese Analyse identifiziert das Jahr, in dem die meisten Fahrzeuge gelistet wurden.
# Dies könnte auf Trends im Gebrauchtwagenmarkt oder auf die Datenbeschaffung hinweisen.

print("\n--- 5.3 Bestes Jahr für den Autoverkauf (nach Anzahl der gelisteten Fahrzeuge) ---")

# Gruppiere nach 'year' und zähle die Anzahl der 'make' Einträge pro Jahr.
# Sortiere dann absteigend, um das Jahr mit den meisten Einträgen zu finden.
fahrzeuge_pro_jahr = df.groupby('year')['make'].count().sort_values(ascending=False)

print("Anzahl der gelisteten Fahrzeuge pro Baujahr (absteigend sortiert):")
display(fahrzeuge_pro_jahr)

# Visualisierung
plt.figure(figsize=(12, 7))
sns.barplot(x=fahrzeuge_pro_jahr.index, y=fahrzeuge_pro_jahr.values, palette='Greens_d')
plt.title('Anzahl der gelisteten Fahrzeuge pro Baujahr', fontsize=16)
plt.xlabel('Baujahr', fontsize=14)
plt.ylabel('Anzahl der Fahrzeuge', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nErkenntnisse:")
print("- Das Jahr mit der höchsten Anzahl an gelisteten Fahrzeugen kann Aufschluss über die Aktivität auf dem Gebrauchtwagenmarkt geben.")
print("- Oft sind dies Jahre, in denen viele Neuwagen verkauft wurden und diese dann als Gebrauchtwagen auf den Markt kamen.")

____________________________________________________________________________________________________________

## 5.4 Entwicklung der Preise über die Jahre

In [ ]:

# Dieser Abschnitt untersucht, wie sich der durchschnittliche Fahrzeugpreis über die Jahre entwickelt hat.
# Dies kann sowohl den durchschnittlichen Neuwagenpreis des jeweiligen Jahres als auch den Wertverlust
# der in diesem Jahr produzierten Fahrzeuge reflektieren.

print("\n--- 5.4 Entwicklung der Preise über die Jahre ---")

# Gruppiere nach 'year' und berechne den Durchschnittspreis für jedes Jahr.
# Sortiere nach Index (Jahr), um einen chronologischen Trend zu sehen.
durchschnittspreis_pro_jahr = df.groupby('year')['price'].mean().sort_index()

print("Durchschnittlicher Preis pro Baujahr:")
display(durchschnittspreis_pro_jahr)

# Visualisierung des Trends
plt.figure(figsize=(12, 7))
sns.lineplot(x=durchschnittspreis_pro_jahr.index, y=durchschnittspreis_pro_jahr.values, marker='o', color='darkblue')
plt.title('Durchschnittlicher Preis pro Baujahr (Trend)', fontsize=16)
plt.xlabel('Baujahr', fontsize=14)
plt.ylabel('Durchschnittlicher Preis (€)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nErkenntnisse:")
print("- Ein klarer Aufwärtstrend bei den durchschnittlichen Preisen jüngerer Baujahre ist zu erwarten, da neuere Fahrzeuge teurer sind und weniger Wertverlust erfahren haben.")
print("- Interessant ist, ob es Jahre gibt, in denen der Preisanstieg besonders steil oder flach ist.")

____________________________________________________________________________________________________________

## 5.5 Kosten Automatikautos mehr als Schaltgetriebe? (Statistischer Test)

In [ ]:

# Diese Analyse verwendet einen T-Test, um zu prüfen, ob ein signifikanter Preisunterschied
# zwischen Fahrzeugen mit Automatikgetriebe und Schaltgetriebe besteht.

print("\n--- 5.5 Kosten Automatikautos mehr als Schaltgetriebe? ---")

# Trenne die Preise basierend auf dem Getriebetyp
automatic_prices = df[df['gear'] == 'Automatic']['price']
manual_prices = df[df['gear'] == 'Manual']['price']
semi_automatic_prices = df[df['gear'] == 'Semi-automatic']['price'] # Falls vorhanden

# Für den T-Test vergleichen wir in der Regel nur zwei Gruppen.
# Hier vergleichen wir Automatik vs. Schaltgetriebe.
# `equal_var=False` wird verwendet, da wir nicht annehmen, dass die Varianzen gleich sind (Welch's T-Test).
# `nan_policy='omit'` stellt sicher, dass fehlende Werte ignoriert werden.
t_stat_auto_manual, p_val_auto_manual = ttest_ind(automatic_prices, manual_prices, equal_var=False, nan_policy='omit')

print(f"Durchschnittspreis Automatik: €{automatic_prices.mean():,.2f}")
print(f"Durchschnittspreis Schaltgetriebe: €{manual_prices.mean():,.2f}")
print(f"T-Statistik (Automatik vs. Schaltgetriebe): {t_stat_auto_manual:.2f}")
print(f"p-Wert (Automatik vs. Schaltgetriebe): {p_val_auto_manual:.3f}")

# ANOVA für mehr als zwei Gruppen (z.B. Automatik, Manuell, Semi-Automatik)
# Da du 'Semi-automatic' hast, ist ANOVA hier noch aussagekräftiger.
if not semi_automatic_prices.empty: # Prüfen, ob Semi-automatic Daten existieren
    f_stat_anova, p_val_anova = f_oneway(automatic_prices, manual_prices, semi_automatic_prices, nan_policy='omit')
    print(f"\nANOVA (Alle Getriebearten):")
    print(f"F-Wert: {f_stat_anova:.2f}, p-Wert: {p_val_anova:.3f}")
else:
    print("\nNicht genügend Daten für 'Semi-automatic', um ANOVA mit allen Getriebearten durchzuführen.")


# Visualisierung der Preisverteilung nach Getriebeart
plt.figure(figsize=(10, 7))
sns.boxplot(x='gear', y='price', data=df, palette='pastel')
plt.title('Preisverteilung nach Getriebeart', fontsize=16)
plt.xlabel('Getriebeart', fontsize=14)
plt.ylabel('Preis (€)', fontsize=14)
plt.tight_layout()
plt.show()


print("\nErkenntnisse:")
if p_val_auto_manual < 0.05:
    print(f"- Der p-Wert ({p_val_auto_manual:.3f}) ist kleiner als 0.05, was darauf hindeutet, dass ein statistisch signifikanter Unterschied zwischen den Durchschnittspreisen von Automatik- und Schaltgetriebe-Fahrzeugen besteht.")
    print(f"- Automatikfahrzeuge sind im Durchschnitt deutlich teurer (€{automatic_prices.mean():,.2f}) als Schaltgetriebe-Fahrzeuge (€{manual_prices.mean():,.2f}).")
else:
    print(f"- Der p-Wert ({p_val_auto_manual:.3f}) ist nicht kleiner als 0.05, was darauf hindeutet, dass kein statistisch signifikanter Unterschied zwischen den Durchschnittspreisen von Automatik- und Schaltgetriebe-Fahrzeugen besteht.")
print("- Der Boxplot visualisiert diesen Unterschied in den Preisverteilungen.")

# ## 6. Vertiefende Korrelationsanalyse

In [ ]:
# Dieser Abschnitt bietet eine tiefere Einsicht in die Beziehungen zwischen numerischen Merkmalen,
# indem verschiedene Korrelationskoeffizienten (Pearson, Spearman, Kendall) berechnet und visualisiert werden.
# Die Wahl des Korrelationskoeffizienten hängt von der Art der Beziehung ab,
# die man untersuchen möchte, und von den Annahmen über die Daten.

print("\n--- 6. Vertiefende Korrelationsanalyse ---")

# Definiere die numerischen Spalten, die in die Korrelationsanalyse einbezogen werden sollen.
# Es ist sinnvoll, 'year' durch 'age' zu ersetzen, da 'age' direkt aus 'year' abgeleitet ist
# und oft die relevantere Information für den Preis darstellt (je älter, desto günstiger).
# 'price_per_mileage' kann auch hinzugefügt werden, wenn es als relevantes Feature betrachtet wird.
numerische_spalten_korr = ['price', 'mileage', 'hp', 'age', 'price_per_mileage'] # Füge hier alle relevanten numerischen Features ein

## --- 6.1 Pearson-Korrelation (Lineare Beziehung) ---

In [ ]:
# Pearson misst die Stärke und Richtung der linearen Beziehung zwischen zwei Variablen.
# Geeignet für metrische, normalverteilte Daten ohne extreme Ausreißer.
print("\n6.1 Pearson-Korrelationsmatrix (Lineare Beziehung):")
korrelationsmatrix_pearson = df[numerische_spalten_korr].corr(method='pearson')
display(korrelationsmatrix_pearson)

plt.figure(figsize=(10, 8))
sns.heatmap(korrelationsmatrix_pearson, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, annot_kws={"size": 10})
plt.title('Pearson-Korrelationsmatrix', fontsize=16)
plt.show()

print("\nErkenntnisse Pearson-Korrelation:")
print("- Werte nahe +1 oder -1 zeigen starke lineare Beziehungen an. Ein Wert von 0 deutet auf keine lineare Beziehung hin.")
print("- Eine hohe positive Korrelation zwischen 'price' und 'hp' ist zu erwarten (mehr PS, höherer Preis).")
print("- Eine negative Korrelation zwischen 'price' und 'mileage'/'age' ist ebenfalls zu erwarten (mehr Kilometer/Alter, niedrigerer Preis).")
print("- Beachte die starke negative Korrelation zwischen 'age' und 'year' (-1.00), da 'age' direkt aus 'year' berechnet wird.")


## --- 6.2 Spearman-Korrelation (Monotone Beziehung) ---

In [ ]:
# Spearman misst die Stärke und Richtung der monotonen Beziehung zwischen zwei Variablen.
# Sie basiert auf den Rängen der Daten und ist robuster gegenüber Ausreißern und nicht-linearen,
# aber monotonen Beziehungen.
print("\n6.2 Spearman-Korrelationsmatrix (Monotone Beziehung):")
korrelationsmatrix_spearman = df[numerische_spalten_korr].corr(method='spearman')
display(korrelationsmatrix_spearman)

plt.figure(figsize=(10, 8))
sns.heatmap(korrelationsmatrix_spearman, annot=True, cmap='viridis', fmt=".2f", linewidths=.5, annot_kws={"size": 10})
plt.title('Spearman-Korrelationsmatrix', fontsize=16)
plt.show()

print("\nErkenntnisse Spearman-Korrelation:")
print("- Wenn der Spearman-Wert sich vom Pearson-Wert unterscheidet, könnte dies auf nicht-lineare, aber immer noch stetige Beziehungen hindeuten.")
print("- Sie ist weniger empfindlich gegenüber Ausreißern und kann auch bei ordinalen Skalen sinnvoll sein.")
print("- Ein hoher Spearman-Koeffizient bedeutet, dass die Ränge der Variablen dazu tendieren, in die gleiche Richtung zu gehen.")

## --- 6.3 Kendall-Korrelation (Rangkorrelation) ---

In [ ]:
# Kendall's Tau ist eine weitere nicht-parametrische Rangkorrelationsmaßnahme,
# die die Ähnlichkeit der Reihenfolge von Datenpaaren misst (konkordante vs. diskordante Paare).
# Sie ist oft kleiner als die Spearman-Korrelation, aber interpretativ ähnlich.
print("\n6.3 Kendall-Korrelationsmatrix (Rangkorrelation):")
korrelationsmatrix_kendall = df[numerische_spalten_korr].corr(method='kendall')
display(korrelationsmatrix_kendall)

plt.figure(figsize=(10, 8))
sns.heatmap(korrelationsmatrix_kendall, annot=True, cmap='plasma', fmt=".2f", linewidths=.5, annot_kws={"size": 10})
plt.title("Kendall's Tau Korrelationsmatrix", fontsize=16)
plt.show()

print("\nErkenntnisse Kendall-Korrelation:")
print("- Kendall's Tau ist oft kleiner als die Spearman-Korrelation, aber sie interpretieren ähnliche Informationen über die Rangordnung.")
print("- Nützlich, wenn die Daten nicht normalverteilt sind oder Ausreißer vorhanden sind.")
print("- Sie misst die Wahrscheinlichkeit, dass zwei Paare in der gleichen Reihenfolge sind (konkordant) minus der Wahrscheinlichkeit, dass sie in unterschiedlicher Reihenfolge sind (diskordant).")

print("\n--- Vertiefende Korrelationsanalyse abgeschlossen ---")

print("\n--- Explorative Datenanalyse (EDA) abgeschlossen ---")

In [ ]:
print("\n--- Explorative Datenanalyse (EDA) abgeschlossen ---")